In [7]:
from PIL import Image
import glob
import torchvision.transforms as transforms
import numpy as np
import csv
import os

In [8]:
class MakeSquare:
    
    def __call__(self, image):
        w, h = image.size
        max_dim = max([w,h])
        horz = (max_dim - w) / 2
        vert = (max_dim - h) / 2
        l_pad = int(np.ceil(horz))
        r_pad = int(np.floor(horz))
        t_pad = int(np.ceil(vert))
        b_pad = int(np.floor(vert))
        return transforms.functional.pad(image, (l_pad, t_pad, r_pad, b_pad))

In [10]:
def get_transformed():
    for cat in ["other", "martin"]:
        images = glob.glob(f'images/raw/{cat}/*.jpg')
        for index, image in enumerate(images):
            im = Image.open(image)
            transform = transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224)
            ])
            augment = transforms.Compose([
                transforms.RandomRotation(5),
                transforms.ColorJitter(brightness=0.5)
            ])
            augmented = transform(augment(im))
            transformed = transform(im)
            transformed.save(f'images/transformed/{cat}/{index}.jpg')
            augmented.save(f'images/transformed/{cat}/{index + len(images)}.jpg')

In [11]:
def create_csv():
    with open('train.csv', 'w') as csvfile:
        writer = csv.writer(csvfile, lineterminator='\n')
        writer.writerow(['image', 'label'])
        for index, cat in enumerate(["other", "martin"]):
            for image in glob.glob(f'images/transformed/{cat}/*.jpg'):
                writer.writerow([image, index])

In [12]:
get_transformed()

In [8]:
create_csv()

In [13]:
for image in glob.glob(f'images/transformed/*/*.jpg'):
    if Image.open(image).size != (224, 224):
        print(image)